In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from tqdm import tqdm
import os

import time
import numpy as np
import pandas as pd


In [2]:
#!cp restaurants.h5 restaurants_copy.h5
restaurants = pd.read_csv('restaurants.csv').set_index('Unnamed: 0')

In [3]:
def get_element(driver, xpath):
    for i in range(5):
        try:
            result = driver.find_element_by_xpath(xpath)
            return result
        except StaleElementReferenceException:
            time.sleep(.5)
    raise StaleElementReferenceException
    
for restaurant_index in tqdm(restaurants.index, total=len(restaurants)):
    
    result_filename = 'reg_reviews/%s.h5' % restaurant_index
    if os.path.exists(result_filename):
        continue
    
    try:
        profile = webdriver.FirefoxProfile()
        profile.set_preference('permissions.default.image', 2)
        #profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
        driver = webdriver.Firefox(firefox_profile=profile)
        driver.get(restaurants.loc[restaurant_index]['restaurant_link'])
    except:
        profile = webdriver.FirefoxProfile()
        profile.set_preference('permissions.default.image', 2)
        #profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
        driver = webdriver.Firefox(firefox_profile=profile)
        driver.get(restaurants.loc[restaurant_index]['restaurant_link'])


    rec_result = []
    while True:
        for i in range(1, len(driver.find_elements_by_xpath('//div[@class="review review--with-sidebar"]'))+1):

            base_xpath = '(//div[@class="review review--with-sidebar"])[%d]' % i
            
            try:
                src = get_element(driver, base_xpath + '//img[@class="photo-box-img"]').get_attribute('src')
                has_photo = 'default_avatars' not in src
            except:
                has_photo = False

            try:
                user_name = get_element(driver, base_xpath + '//li[@class="user-name"]').text
            except:
                user_name = ''
            try:
                localtion = get_element(driver, base_xpath + '//li[@class="user-location responsive-hidden-small"]').text
            except:
                localtion = ''
            try:
                friends = get_element(driver, base_xpath + '//li[@class="friend-count responsive-small-display-inline-block"]').text
                friends = int(friends.split(' ')[0])
            except:
                friends = 0
            try:
                reviews = get_element(driver, base_xpath + '//li[@class="review-count responsive-small-display-inline-block"]').text
                reviews = int(reviews.split(' ')[0])
            except:
                reviews = -1
            try:
                photos = get_element(driver, base_xpath + '//li[@class="photo-count responsive-small-display-inline-block"]').text
                photos = int(photos.split(' ')[0])
            except:
                photos = 0
            try:
                rating = get_element(driver, base_xpath + '//div[contains(@class, "i-stars")]').get_attribute('class')
                rating = int(rating.split(' ')[1][-1])
            except:
                rating = -1
            try:
                date = get_element(driver, base_xpath + '//span[@class="rating-qualifier"]').text
            except:
                date = ''
            try:
                review = get_element(driver, base_xpath + '//span[@class="js-content-toggleable hidden"]').get_attribute('innerHTML')
            except:
                try:
                    review = get_element(driver, base_xpath + '//span[@class="js-content-toggleable"]').get_attribute('innerHTML')
                except:
                    try:
                        review = get_element(driver, base_xpath + '//div[@class="review-content"]/div[2]').text
                    except:
                        try:
                            review = get_element(driver, base_xpath + '//div[@class="review-content"]/p').text
                        except:
                            review = -1


            try:
                useful = get_element(driver, base_xpath + '//a[@class="ybtn ybtn--small useful js-analytics-click"]').text
                useful = int(useful.split(' ')[1])
            except:
                useful = 0

            try:
                funny = get_element(driver, base_xpath + '//a[@class="ybtn ybtn--small funny js-analytics-click"]').text
                funny = int(funny.split(' ')[1])
            except:
                funny = 0

            try:
                cool = get_element(driver, base_xpath + '//a[@class="ybtn ybtn--small cool js-analytics-click"]').text
                cool = int(cool.split(' ')[1])
            except:
                cool = 0

            rec_result.append(dict(
                restaurant_id=restaurant_index,
                has_photo=has_photo,
                user_name=user_name,
                localtion=localtion,
                friends=friends,
                reviews=reviews,
                photos=photos,
                rating=rating,
                date=date,
                review=review,
                useful=useful,
                funny=funny,
                cool=cool
            ))

        try:
            driver.find_element_by_xpath('//span[@class="pagination-label responsive-hidden-small pagination-links_anchor"]').click()
        except:
            break

    rec_result = pd.DataFrame(rec_result)
    rec_result.to_hdf(result_filename, key='data', mode='w')
    driver.close()

  0%|          | 0/663 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['date', 'localtion', 'review', 'user_name']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
 95%|█████████▍| 627/663 [00:40<00:02, 15.67it/s]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['date', 'localtion', 'review', 'user_name']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
 95%|█████████▍| 628/663 [00:48<00:02, 12.97it/s]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object t

 98%|█████████▊| 652/663 [1:34:32<01:35,  8.70s/it]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['date', 'localtion', 'review', 'user_name']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
 98%|█████████▊| 653/663 [1:39:27<01:31,  9.14s/it]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['date', 'localtion', 'review', 'user_name']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
 99%|█████████▊| 654/663 [1:50:57<01:31, 10.18s/it]/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will